# Using Xpress in python

Before we can start using the Xpress package that we have just installed we will need to begin by importing it.

In [1]:
import xpress as xp
import pandas as pd
import numpy as np

Lets begin by writing some code to solve the simple linear programming problem

min $z = x +y$

s.t. 

$2x + y \ge 1$,

$x +2y \ge 1$,
     
$ -1 \ge y \le 1$.

The first thing we need to do is create a problem. 

In [2]:
# Create a problem called myproblem

prob = xp.problem(name='myproblem')

Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/home/codespace/.python/current/lib/python3.10/site-packages/xpress/license/community-xpauth.xpr')


We are now ready to define our variables using the xp.var function:

`var(name, lb, ub, threshold, vartype)`
 
The `name`, `lb` and `up` arguments are self explanatory. The `threshold` argument is the threshold for semi-continuous, semi-integer, and partially integer variables and the `vartype` argument is used to set the variable type (`continuous`, `binary`, `integer` ect.).

Once the variables are defined we will add them to our problem.

In [3]:
# Create the x and y variables
x = xp.var('x')
y = xp.var('y',lb=-1, ub=1)

# Add the variables to the problem

prob.addVariable(x,y)

We can now begin to add our constraints. To do this we will use the xp.constraints function:

In [4]:
prob.addConstraint(2*x + y >= 1)
prob.addConstraint(x + 2*y >= 1)

We can now define our objective function using the `xp.setObjective()` function:

`setObjective(objective, sense)`

The `objective` is the expression defining the new objective and `sense` is either `xp.minimize` or `xp.maximize`(note that it must be American spelling).

In [5]:
prob.setObjective(x + y, sense = xp.minimize)

We can now solve the problem!

In [6]:
prob.solve()

FICO Xpress v9.2.5, Community, solve started 22:09:42, Nov 22, 2023
Heap usage: 388KB (peak 388KB, 103KB system)
Minimizing LP myproblem using up to 2 threads and up to 7929MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         2 rows            2 cols            4 elements
Presolve finished in 0 seconds
Heap usage: 388KB (peak 403KB, 103KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 5.00e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 5.00e-01,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         -1.000000      D      2     0        .000000     0
     2           .666667      D      0     0        .000000

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

It might now be a good idea to print out the results.

In [7]:
print(f'x =  {prob.getSolution(x)}')
print(f'y =  {prob.getSolution(y)}') 
print(f'The objective function value is {prob.getObjVal()}') 

x =  0.3333333333333333
y =  0.33333333333333337
The objective function value is 0.6666666666666667


This is a very simple example, lets try to solve a more complex problem. Remember the sailco example from the week 2 lecture.

$$\begin{array}{lll}
		 \min & \displaystyle\sum_{t=1}^4 400x_{1t} + \sum_{t=1}^4 35x_{2t} + \sum_{t=1}^4 50i_{1t} + \sum_{t=1}^4 2i_{2t}\\
		 \text{s.t.} & 20x_{1t} + 3x_{2t} \leq 1860, & t=1,2,3,4,\\
		 & y_{it}\geq d_{i,t}, &  t=1,2,3,4,\\
		 & i_{p1} = x_{p1} - y_{p1}, & p=1,2,\\
		 & i_{pt} = i_{p,t-1} + x_{pt} - y_{pt}, & p=1,2, \ t=2,3,4,\\
		 & i_{pt}, x_{pt}, y_{pt} \geq 0, & p=1,2, \ t=1,2,3,4
		 \end{array}$$
with demand 


|           | Spring | Summer | Autumn | Winter |
|-----------|--------|--------|--------|--------|
| Sailboat  | 40     | 60     | 75     | 25     |
| Surfboard | 190    | 350    | 130    | 20     |

lets solve this problem. Lets begin by creating a numpy array containing our demand data.

In [8]:
# Creating the demand matrix

demand = np.array([[ 40, 60, 75, 25],[190, 350, 130, 20]])
print(demand)

# Creating lists with seasons and products

product_names = ['Sailboat', 'Surfboard']
season_names = ['Spring', 'Summer', 'Autumn', 'Winter']

[[ 40  60  75  25]
 [190 350 130  20]]


Now lets define our index sets.

In [9]:
# Defining the index sets

number_of_products = 2
number_of_periods = 4
Products = range(number_of_products)
Periods = range(number_of_periods)

Now lets define our problem and decision variables using numpy arrays.

In [10]:
# Define the problem
prob = xp.problem('Sailco')

# Define our decision variable as a numpy array
# We use the reshape function to make the numpy array a matrix
make = np.array([xp.var( name='make_{0}_{1}'.format(i+1,j+1))
                    for i in Products for j in Periods], dtype=xp.npvar).reshape(number_of_products,number_of_periods)
sell = np.array([xp.var( name='sell_{0}_{1}'.format(i+1,j+1))
                    for i in Products for j in Periods], dtype=xp.npvar).reshape(number_of_products,number_of_periods)
inventory = np.array([xp.var( name='inventory_{0}_{1}'.format(i+1,j+1))
                    for i in Products for j in Periods], dtype=xp.npvar).reshape(number_of_products,number_of_periods)

print(make)

prob.addVariable(make,sell,inventory)

[[make_1_1 make_1_2 make_1_3 make_1_4]
 [make_2_1 make_2_2 make_2_3 make_2_4]]


Lets add our constraints

In [11]:
# Labour restriction
prob.addConstraint(20*make[0,t] + 3*make[1,t] <= 1860 for t in Periods)

# Demand constraints
prob.addConstraint(sell[i,t] >= demand[i,t] for i in Products for t in Periods)

# Inventory ballance for all times except first time period
prob.addConstraint(inventory[i,t] == inventory[i,t-1] + make[i,t] - sell[i,t] for i in Products for t in Periods if t != 0)


# Inventory ballance for first time period
prob.addConstraint(inventory[i,0] == make[i,0] -sell[i,0] for i in Products)

Now lets add our objective function. Note how we use the the sum function contained in the Xpress module rather than the native python operator. This is for efficiency reasons 

In [12]:
# Define the objective function

prob.setObjective(400*xp.Sum(make[0,t] for t in Periods) + 35*xp.Sum(make[1,t] for t in Periods) + 
                  50*xp.Sum(inventory[0,t] for t in Periods) +  2*xp.Sum(inventory[1,t] for t in Periods), 
                  sense = xp.minimize)

Sometimes it is good to check our problem before we solve it, we can do this by saving the corresponding `lp` file. This is very useful for debugging.

In [13]:
prob.write("problem","lp")

In [14]:
prob.solve()

FICO Xpress v9.2.5, Community, solve started 22:09:42, Nov 22, 2023
Heap usage: 395KB (peak 427KB, 106KB system)
Minimizing LP noname using up to 2 threads and up to 7929MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        20 rows           24 cols           46 elements
Presolved problem has:
        10 rows           12 cols           24 elements
Presolve finished in 0 seconds
Heap usage: 397KB (peak 427KB, 106KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+01] / [ 1.88e-01,  1.25e+00]
  RHS and bounds [min,max] : [ 2.00e+01,  1.86e+03] / [ 2.00e+01,  3.50e+02]
  Objective      [min,max] : [ 2.00e+00,  4.00e+02] / [ 2.00e+00,  4.50e+02]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0       50383.32319      D      6     0        .000000     0
     9       104450.0000      D      0     0        .000000   

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

Lets add our solutions to data frames. 

In [15]:
# Add the solutions into a dataframe

make_df = pd.DataFrame(data = prob.getSolution(make), index = product_names, columns = season_names)
sell_df = pd.DataFrame(data = prob.getSolution(sell), index = product_names, columns = season_names)
inventory_df = pd.DataFrame(data = prob.getSolution(inventory), index = product_names, columns = season_names)

make_df = make_df.style.set_caption('Make')
sell_df = sell_df.style.set_caption('Sell')
inventory_df = inventory_df.style.set_caption('Inventory')

display(make_df)
display(sell_df)
display(inventory_df)

,Spring,Summer,Autumn,Winter
Sailboat,40.000000,60.000000,75.000000,25.000000
Surfboard,330.000000,220.000000,120.000000,20.000000


,Spring,Summer,Autumn,Winter
Sailboat,40.000000,60.000000,75.000000,25.000000
Surfboard,190.000000,350.000000,130.000000,20.000000


,Spring,Summer,Autumn,Winter
Sailboat,0.000000,0.000000,0.000000,0.000000
Surfboard,140.000000,10.000000,0.000000,0.000000


In some cases we might want to use dictionaries to define our decision variables. Lets consider the sudoku problem from week. We will start off by defining the dimension of the sudoku puzzle and by creating the initial puzzle.

In [16]:
# The input is a starting grid where the unknown numbers are replaced by zero
# q is the dimension of the sub blocks
q = 3

starting_grid = \
 [[8, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 3, 6, 0, 0, 0, 0, 0],
  [0, 7, 0, 0, 9, 0, 2, 0, 0],
  [0, 5, 0, 0, 0, 7, 0, 0, 0],
  [0, 0, 0, 0, 4, 5, 7, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 3, 0],
  [0, 0, 1, 0, 0, 0, 0, 6, 8],
  [0, 0, 8, 5, 0, 0, 0, 1, 0],
  [0, 9, 0, 0, 0, 0, 4, 0, 0]]

We will now define our decision variables $x_{i,j,k}$ as a dictionary.

In [17]:
n = q**2  # the size must be the square of the size of the subgrids
N = range(n)

x = {(i, j, k): xp.var(vartype=xp.binary, name='x{0}_{1}_{2}'.format(i, j, k))
     for i in N for j in N for k in N}

print(x)

{(0, 0, 0): x0_0_0, (0, 0, 1): x0_0_1, (0, 0, 2): x0_0_2, (0, 0, 3): x0_0_3, (0, 0, 4): x0_0_4, (0, 0, 5): x0_0_5, (0, 0, 6): x0_0_6, (0, 0, 7): x0_0_7, (0, 0, 8): x0_0_8, (0, 1, 0): x0_1_0, (0, 1, 1): x0_1_1, (0, 1, 2): x0_1_2, (0, 1, 3): x0_1_3, (0, 1, 4): x0_1_4, (0, 1, 5): x0_1_5, (0, 1, 6): x0_1_6, (0, 1, 7): x0_1_7, (0, 1, 8): x0_1_8, (0, 2, 0): x0_2_0, (0, 2, 1): x0_2_1, (0, 2, 2): x0_2_2, (0, 2, 3): x0_2_3, (0, 2, 4): x0_2_4, (0, 2, 5): x0_2_5, (0, 2, 6): x0_2_6, (0, 2, 7): x0_2_7, (0, 2, 8): x0_2_8, (0, 3, 0): x0_3_0, (0, 3, 1): x0_3_1, (0, 3, 2): x0_3_2, (0, 3, 3): x0_3_3, (0, 3, 4): x0_3_4, (0, 3, 5): x0_3_5, (0, 3, 6): x0_3_6, (0, 3, 7): x0_3_7, (0, 3, 8): x0_3_8, (0, 4, 0): x0_4_0, (0, 4, 1): x0_4_1, (0, 4, 2): x0_4_2, (0, 4, 3): x0_4_3, (0, 4, 4): x0_4_4, (0, 4, 5): x0_4_5, (0, 4, 6): x0_4_6, (0, 4, 7): x0_4_7, (0, 4, 8): x0_4_8, (0, 5, 0): x0_5_0, (0, 5, 1): x0_5_1, (0, 5, 2): x0_5_2, (0, 5, 3): x0_5_3, (0, 5, 4): x0_5_4, (0, 5, 5): x0_5_5, (0, 5, 6): x0_5_6, (0, 5, 7): 

Before we define our constraints lest define the box index set.

In [18]:
# define all q^2 subgrids
subgrids = {(h, l): [(i, j) for i in range(q*h, q*h + q)
            for j in range(q*l, q*l + q)]
            for h in range(q)
            for l in range(q)}
print(subgrids)

{(0, 0): [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)], (0, 1): [(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)], (0, 2): [(0, 6), (0, 7), (0, 8), (1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8)], (1, 0): [(3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2)], (1, 1): [(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)], (1, 2): [(3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (5, 6), (5, 7), (5, 8)], (2, 0): [(6, 0), (6, 1), (6, 2), (7, 0), (7, 1), (7, 2), (8, 0), (8, 1), (8, 2)], (2, 1): [(6, 3), (6, 4), (6, 5), (7, 3), (7, 4), (7, 5), (8, 3), (8, 4), (8, 5)], (2, 2): [(6, 6), (6, 7), (6, 8), (7, 6), (7, 7), (7, 8), (8, 6), (8, 7), (8, 8)]}


Now lets define all the constraints

In [19]:
vertical = [xp.Sum(x[i, j, k] for i in N) == 1 for j in N for k in N]
horizontal = [xp.Sum(x[i, j, k] for j in N) == 1 for i in N for k in N]
subgrid = [xp.Sum(x[i, j, k] for (i, j) in subgrids[h, l]) == 1
           for (h, l) in subgrids.keys() for k in N]

# Assign exactly one number to each cell

assign = [xp.Sum(x[i, j, k] for k in N) == 1 for i in N for j in N]

# Fix those variables that are non-zero in the input grid

init = [x[i, j, k] == 1 for k in N for i in N for j in N
        if starting_grid[i][j] == k+1]
print(vertical)

[R23, R24, R25, R26, R27, R28, R29, R30, R31, R32, R33, R34, R35, R36, R37, R38, R39, R40, R41, R42, R43, R44, R45, R46, R47, R48, R49, R50, R51, R52, R53, R54, R55, R56, R57, R58, R59, R60, R61, R62, R63, R64, R65, R66, R67, R68, R69, R70, R71, R72, R73, R74, R75, R76, R77, R78, R79, R80, R81, R82, R83, R84, R85, R86, R87, R88, R89, R90, R91, R92, R93, R94, R95, R96, R97, R98, R99, R100, R101, R102, R103]


We are now ready to define the problem, add our constraints and decision variables.

In [20]:
# Define the problem
p = xp.problem()

# Include decision variables
p.addVariable(x)

# Include constraints
p.addConstraint(vertical, horizontal, subgrid, assign, init)

Now lets solve the problem!

In [21]:
p.solve()

print('Solution:')

for i in N:
    for j in N:
        l = [k for k in N if p.getSolution(x[i, j, k]) >= 0.5]
        assert(len(l) == 1)
        print('{0:2d}'.format(1 + l[0]), end='', sep='')
    print('')

FICO Xpress v9.2.5, Community, solve started 22:09:43, Nov 22, 2023
Heap usage: 584KB (peak 584KB, 301KB system)
Minimizing MILP noname using up to 2 threads and up to 7929MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
       345 rows          729 cols         2937 elements       729 entities
Presolved problem has:
       140 rows          200 cols          595 elements       200 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 696KB (peak 1021KB, 301KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 6.4GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve, 